In [11]:
import os
import pandas as pd
import yaml
import copy
import random
from pathlib import Path
import matplotlib.pyplot as plt
from copy import deepcopy

import aot

In [12]:
base_dir = Path(aot.__path__[0])
core_expt_yaml_path = base_dir / "experiment/core_exp_settings.yml"
stimuli_temp_path = base_dir / "experiment/stimuli_settings_temp.yml"
core_settings = yaml.load(open(core_expt_yaml_path), Loader=yaml.FullLoader)
stimuli_settings_temp = yaml.load(open(stimuli_temp_path), Loader=yaml.FullLoader)  

settings_root_path = base_dir / core_settings["paths"]["settings_path"] 
video_db_path = base_dir / 'data/videos/database_originals.tsv'
video_db = pd.read_csv(video_db_path, sep='\t')

main_settings_dir = base_dir / core_settings["paths"]["settings_path"] / "main"
memory_settings_dir = base_dir / core_settings["paths"]["settings_path"] / "memory"
subject_number = core_settings['various']['subject_number']
run_number = core_settings['various']['run_number']
session_number = core_settings['various']['session_number']
unique_video_number_per_session = core_settings['various']['unique_video_number_per_session']
img_number_per_session = unique_video_number_per_session



imagenet_folder = '/Users/shufanzhang/Documents/PhD/Arrow_of_time/arrow_of_time/imagenet_sub'
imagenet_pictures = os.listdir(imagenet_folder)



unused_videos = []
for i in range(1, len(video_db)):
    if video_db['grade'][i] != 'j' and video_db['grade'][i] != 'NA': 
        unused_videos.append(video_db['video_name'][i])
print("unused videos number: ", len(unused_videos))

imgs_from_unused_videos = []
for i in range(len(unused_videos)):
    imgs_from_unused_videos.append(unused_videos[i].replace('.mp4', '.png'))
imgs_from_unused_videos = ["S_"+i for i in imgs_from_unused_videos]+imagenet_pictures
print("unused images number: ", len(imgs_from_unused_videos))

unused videos number:  528
unused images number:  2766


In [13]:
def gen_memory_design(subject,session):
    print(f"subject: {subject}, session: {session}")
    #read main experiment settings
    main_runs_settings = []
    main_runs_videos = []
    main_runs_videos_name = []#raw video name without .mp4
    for ind in range(run_number):
        run = ind + 1
        runfile = f"experiment_settings_sub_{str(subject).zfill(2)}_ses_{str(session).zfill(2)}_run_{str(run).zfill(2)}.yml"
        runfile = main_settings_dir / runfile
        runsetting = yaml.load(open(runfile), Loader=yaml.FullLoader)
        main_runs_settings.append(runsetting)
    for runsetting in main_runs_settings:
        raw_video_list = runsetting["stimuli"]["movie_files"]
        video_list = [video for video in raw_video_list if video != "blank"]
        #print(video_list)
        main_runs_videos.append(video_list)
    for videos in main_runs_videos:
        name_list= [video.split(".")[0] for video in videos]
        main_runs_videos_name.append(name_list)
        #print(name_list)
    #flatten the videos list
    flat_video_names = [item for sublist in main_runs_videos_name for item in sublist]
    #remove repeated videos
    flat_video_names = list(set(flat_video_names))
    #print(flat_video_names)
    imgnames = [video + ".png" for video in flat_video_names]
    print(imgnames)
    #save imgnames as settings for memory experiment yaml file
    settings = settings = copy.deepcopy(stimuli_settings_temp)

    half_img_num = int((img_number_per_session)/2)
    #sample half_img_num images from imgnames
    imgnames_used = random.sample(imgnames, half_img_num)
    imgnames_unused = random.sample(imgs_from_unused_videos, half_img_num)
    imgnames_final = imgnames_used + imgnames_unused
    random.shuffle(imgnames_final)

    settings["stimuli"]["picture_files"] = imgnames_final##not right. should subsample from imgnames and add some img from unused videos
    print(settings)
    outfile = f"experiment_settings_sub_{str(subject).zfill(2)}_ses_{str(session).zfill(2)}_run_01.yml"
    outfile = memory_settings_dir / outfile
    with open(outfile, "w") as f:
        yaml.dump(settings, f)


In [14]:
def gen_memory_desing_for_all():
    for sub in range(subject_number):
        sub = sub + 1
        for ses in range(session_number):
            ses = ses + 1
            gen_memory_design(sub, ses)       

In [15]:
gen_memory_desing_for_all()

subject: 1, session: 1
['S_yt-4BAsE47kJNk_88.png', 'S_yt-RKo978aBdzw_173.png', 'R_S_yt-TNtO8RPd3iE_25.png', 'S_yt-nlfwzxlwRfk_12.png', 'S_yt-qiPhBL4lBr0_44.png', 'S_yt-8aJDzfAaArQ_75.png', 'S_ILSVRC2015_train_00405001.png', 'S_yt-GWgG8PVM1iw_34.png', 'S_yt-4i1b6NQ9NKs_5.png', 'R_S_Pexels_Videos_2795736.png', 'R_S_yt-SS7pSyb0_d0_32.png', 'R_S_yt-qiPhBL4lBr0_44.png', 'R_S_yt-EApB0DJ3ECg_66.png', 'S_production_ID_5144823.png', 'S_pexels-c-technical-6763571.png', 'S_flickr-southeasternstar-3464778323_10.png', 'S_pexels-yaroslav-shuraev-8091356.png', 'R_S_yt-IPiZqn4dG9k_9.png', 'R_S_ILSVRC2015_train_00373000.png', 'S_yt-_68FWLGgPYE_25.png', 'S_yt-lkpfs_4qAGE_50.png', 'R_S_pexels-yaroslav-shuraev-8091356.png', 'S_yt-7v3kHYBe7-E_8.png', 'R_S_pexels-mart-production-7279749.png', 'R_S_yt-tVfYnm81ywo_82.png', 'R_S_yt-l6WPyKg_3is_154.png', 'S_59eT4XDXNno_25.png', 'S_DV6V38rplag_215.png', 'S_yt-hWXr5bv2_uA_48.png', 'S_yt-J1dw4AznwmQ_7.png', 'R_S_yt-X6ivH9SqGu8_306.png', 'S_yt-TNtO8RPd3iE_25.png', 